In [1]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from matplotlib.patches import Patch
import torch
import pandas as pd
import copy
from voltron.option_utils import GetTradingDays, GetTrainingData, Pricer, FindLastTradingDays
from scipy.optimize import minimize

In [2]:
SPY = pd.read_csv("./data/SPY_prices.csv")
SPY['Date'] = pd.to_datetime(SPY['Date'])

years = np.arange(2009, 2018)
opts = pd.DataFrame()
for year in years:
    dat = pd.read_csv("./data/SPY_" + str(year) + ".csv")
    dat = dat[dat.type == "call"]
    quotedate = dat.quotedate.unique()[0]
    dat = dat[dat.quotedate == quotedate]    
    opts = pd.concat((opts, dat), ignore_index=True)

# exps = [] 
# for idx, row in opts.iterrows():
#     eday = row.expiration
#     exps.append(SPY[SPY.Date == FindLastTradingDays(SPY, [pd.Timestamp(eday)])[0]].Close.item())
# opts['exp_price'] = exps

/Users/gregorybenton/miniconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (19) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


## Torch Attempt

In [3]:
ivol = torch.tensor(opts.impliedvol.to_numpy())
Fs = torch.tensor(opts.underlying_last.to_numpy())
Ks = torch.tensor(opts.strike.to_numpy())
qdays = pd.to_datetime(opts.quotedate).dt.date.to_numpy()
edays = pd.to_datetime(opts.expiration).dt.date.to_numpy()
Ts = torch.tensor(([np.busday_count(qd, ed)/252. for qd, ed in zip(qdays, edays)]))

In [4]:
def BlackVol(pars, K, f, T):
    alpha = torch.exp(pars[0][0])
    rho = 2 * torch.sigmoid(pars[0][1]) - 1.
    v = torch.exp(pars[0][2])
    beta = 1.
    num = 1 + (alpha**2 * (1-beta)**2/(24 * (f*K)**(1-beta)) + 0.25 * rho*beta*v*alpha/((f*K)**(0.5*(1-beta))) + v**2*(2-3*rho**2)/24)*T
    num*= alpha
    
    denom = (f*K)**(0.5*(1-beta)) * (1 + (1-beta)**2/24 * torch.log(f/K)**2 + (1-beta)**4/1920 * torch.log(f/K)**4)
    
    z = v/alpha * (f*K)**(0.5*(1-beta)) * np.log(f/K)
    xi_z = torch.log((torch.sqrt(1 - 2 * rho * z + z**2) + z - rho)/(1-rho))
    
    return num/denom * z/xi_z

In [5]:
def MinVol(pars):
    return torch.mean((ivol - BlackVol(pars, Ks, Fs, Ts)).pow(2))

In [6]:
pars = [torch.tensor([-1., -5., -3.], requires_grad=True)]
opt = torch.optim.SGD(pars, lr=0.1)

In [7]:
iters = 1000
stored_pars = torch.zeros(iters, 3)
losses = []
for e in range(iters):
    stored_pars[e, :] = pars[0]
    loss = MinVol(pars)
    opt.zero_grad()
    loss.backward()
    losses.append(loss.item())
    opt.step() 

In [8]:
plt.plot(stored_pars.detach())

In [9]:
plt.plot(losses)

# Running SABR

In [10]:
import numpy as np
import torch
import pandas as pd
import copy
from voltron.option_utils import GetTradingDays, GetTrainingData, Pricer, FindLastTradingDays
from scipy.optimize import minimize

def BlackVol(pars, K, f, T):
    alpha = torch.exp(pars[0][0]) ## v0
    rho = 2 * torch.sigmoid(pars[0][1]) - 1. ##rho
    v = torch.exp(pars[0][2]) ## "sigma" 
    beta = 1.
    num = 1 + (alpha**2 * (1-beta)**2/(24 * (f*K)**(1-beta)) +\
               0.25 * rho*beta*v*alpha/((f*K)**(0.5*(1-beta))) +\
               v**2*(2-3*rho**2)/24)*T
    num*= alpha
    
    denom = (f*K)**(0.5*(1-beta)) * (1 + (1-beta)**2/24 * torch.log(f/K)**2 +\
                                     (1-beta)**4/1920 * torch.log(f/K)**4)
    
    z = v/alpha * (f*K)**(0.5*(1-beta)) * np.log(f/K)
    xi_z = torch.log((torch.sqrt(1 - 2 * rho * z + z**2) + z - rho)/(1-rho))
    
    return num/denom * z/xi_z

def MinVol(pars, Ks, Fs, Ts, ivol):
    return torch.mean((ivol - BlackVol(pars, Ks, Fs, Ts)).pow(2))

def Calibrate(Fs, Ks, Ts, ivol, iters=1000):
    pars = [torch.tensor([-1., -5., -3.], requires_grad=True)]
    opt = torch.optim.SGD(pars, lr=0.1)
    stored_pars = torch.zeros(iters, 3)
    losses = []
    for e in range(iters):
        stored_pars[e, :] = pars[0]
        loss = MinVol(pars, Ks, Fs, Ts, ivol)
        opt.zero_grad()
        loss.backward()
        losses.append(loss.item())
        opt.step() 
    
    return pars[0].detach().numpy()

def SABRSim(Np, Nt, S0, V0, sigma, rho, dt=1./252.):
    dW = np.random.randn(Nt+1, Np) * np.sqrt(dt)
    dZ = rho * dW + np.sqrt(1-rho**2) * np.random.randn(Nt+1, Np) * np.sqrt(dt)
    
    S = np.zeros((Nt+1, Np))
    S[0] = S0
    V = np.zeros((Nt+1, Np))
    V[0] = V0
    
    for t in range(Nt):
        S[t+1] = S[t] + V[t]*S[t]*dW[t]
        V[t+1] = V[t] + sigma*V[t]*dZ[t]
        
    return S[1:]

In [11]:
logger = []
full_logger = []
SPY = pd.read_csv("./data/SPY_prices.csv")
SPY['Date'] = pd.to_datetime(SPY['Date'])
Np = 10000
ntrain = 252
year = 2012
options = pd.read_csv("./data/SPY_" + str(year) + ".csv")
options.expiration = pd.to_datetime(options.expiration)
options.quotedate = pd.to_datetime(options.quotedate)
qday = options.quotedate.unique()[0]
quote_price = SPY[SPY['Date']==qday].Close.item()
options = options[(options.quotedate == qday) & (options.type=='call')]
edays = options.expiration.sort_values().unique()
testdays = (edays - qday)/np.timedelta64(1, "D")
edays = edays[(testdays > 100) & (testdays < 365)]
lastdays = FindLastTradingDays(SPY, edays)
ntests = np.array([GetTradingDays(SPY, qday, 
                                  pd.Timestamp(ld)) for ld in lastdays])
fulltest = ntests[-1]
train_y = torch.FloatTensor(GetTrainingData(SPY, qday, ntrain).to_numpy())
test_y = torch.FloatTensor(GetTrainingData(SPY, 
                                           pd.Timestamp(lastdays[-1]),
                                           fulltest).to_numpy())
full_x = torch.arange(ntrain+fulltest).type(torch.FloatTensor)
full_x = full_x/252.
train_x = full_x[:ntrain]
test_x = full_x[ntrain:]

## extract data for calibration ##
ivol = torch.tensor(options.impliedvol.to_numpy())
Fs = torch.tensor(options.underlying_last.to_numpy())
Ks = torch.tensor(options.strike.to_numpy())
starts = options.quotedate.dt.date.to_numpy()
ends = options.expiration.dt.date.to_numpy()
Ts = torch.tensor(([np.busday_count(qd, ed)/252. for qd, ed in zip(starts, ends)]))

pars = Calibrate(Fs, Ks, Ts, ivol)
v0 = np.exp(pars[0])
1/(1 + np.exp(-pars[1]))
rho = (2/(1 + np.exp(-pars[1])) - 1.)
sigma = np.exp(pars[2])
px_paths = SABRSim(Np, fulltest, quote_price, v0, sigma, rho)
px_samples = torch.tensor(px_paths[ntests-1])

In [12]:
option_output = Pricer(torch.tensor(px_paths), options, edays, test_y[ntests-1],
                       quote_price)

In [13]:
option_output

,Expiry,Strike,Bid,Ask,Voltron,Return,ExpClose,QuoteClose,Year,Sample_Percentile
0,2013-03-16,20.0,121.39,121.61,126.884545,136.729996,156.729996,141.449997,2013,0.781553
1,2013-03-16,25.0,116.39,116.61,121.884545,131.729996,156.729996,141.449997,2013,0.781553
2,2013-03-16,30.0,111.39,111.61,116.884545,126.729996,156.729996,141.449997,2013,0.781553
3,2013-03-16,35.0,106.39,106.61,111.884545,121.729996,156.729996,141.449997,2013,0.781553
4,2013-03-16,40.0,101.39,101.61,106.884545,116.729996,156.729996,141.449997,2013,0.781553
...,...,...,...,...,...,...,...,...,...,...
651,2013-09-30,169.0,0.33,0.42,0.000000,0.690002,169.690002,141.449997,2013,1.000000
652,2013-09-30,170.0,0.28,0.37,0.000000,0.000000,169.690002,141.449997,2013,1.000000
653,2013-09-30,175.0,0.14,0.21,0.000000,0.000000,169.690002,141.449997,2013,1.000000
654,2013-09-30,180.0,0.08,0.12,0.000000,0.000000,169.690002,141.449997,2013,1.000000


In [14]:
plt.plot(train_x, train_y)
plt.plot(test_x, test_y)
plt.plot(test_x, px_paths[:, :20], c='gray', alpha=0.5);

In [15]:
px_paths[:, :20].shape

(206, 20)

In [16]:
px_paths.shape

(206, 10000)